In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split

import xgboost as xgb

from sklearn.metrics import accuracy_score

from bayes_opt import BayesianOptimization


In [3]:
# Load Data

file_name = "../data/train_preprocessed2.csv"
train_df = pd.read_csv(file_name, low_memory = False)

train_df.head()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,...,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,0.0,...,0,0,2,0,0,0,0,0,0,1
3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,0.0,...,0,0,2,0,0,0,0,0,0,1
4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
#Setup data : Divide Test and Train set

array = train_df.values

data = array[:, 0:70]
target = array[:, 70]

data, target

seed = 7
test_size = 0.2

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = test_size, random_state = seed)

In [10]:
# set XGB Model -> parameters set default

model = xgb.XGBClassifier()
model.fit(data_train, target_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
#Make predictions with XGBoost Model
target_pred = model.predict(data_test)
predictions = [round(value) for value in target_pred]
accuracy = accuracy_score(target_test, predictions)
print("Accuracy : %.2f%%" %(accuracy * 100))


Accuracy : 88.46%


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
# Making a Model function for bayesian optimization

def XGB_Train_Model(max_depth, subsample, min_child_weight, gamma, colsample_bytree) : 
    xgb_params = {
        'n_trees' : 250,
        'eta' : 0.01,
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'objective' : 'reg:linear', 
        'base_score' : np.mean(target),
        'silent' : 1,
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(data_train, target_train)
    target_pred = model.predict(data_test)
    predictions = [round(value) for value in target_pred]
    accuracy = accuracy_score(target_test, predictions)
    return accuracy

In [27]:
xgb_params = {
    'min_child_weight' : (1, 20), 
    'gamma' : (0, 10), 
    'subsample' : (0.5, 1),
    'colsample_bytree' : (0.1, 1),
    'max_depth' : (2, 10)
}

xgb_bayesOPT = BayesianOptimization(XGB_Train_Model, xgb_params)
xgb_bayesOPT.maximize(init_points = 5, n_iter = 25)


Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


    1 | 00m00s |    0.85591 |             0.4668 |    8.9591 |      2.1515 |             8.1634 |      0.5856 | 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


    2 | 00m00s |    0.83984 |             0.1248 |    9.9767 |      6.0149 |            10.8635 |      0.7828 | 


...

   28 | 00m17s |    0.88691 |             0.2235 |    1.7537 |      9.9879 |            11.8564 |      0.5255 | 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-8.90417723e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   29 | 00m17s |    0.86567 |             0.1000 |    0.0000 |     10.0000 |            20.0000 |      0.5000 | 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


   30 | 00m17s |    0.86797 |             1.0000 |    3.4764 |      2.0000 |            14.6076 |      1.0000 | 
